In [1]:
import csv
def load_sequences_from_csv(filename):
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        return [[tuple(map(float, x.strip('()').split(','))) for x in row] for row in reader]

# Example usage:
sequences = load_sequences_from_csv('sequences_b.csv')


In [2]:
seq = sequences

In [3]:
from collections import defaultdict
from itertools import combinations

def generate_subsequences(transaction):
    """Generate all possible subsequences of any length in a transaction"""
    subsequences = []
    for length in range(1, len(transaction) + 1):  # For lengths from 1 to the length of the transaction
        subsequences.extend(combinations(transaction, length))  # Get all combinations of the given length
    return subsequences

def generate_unique_subsequences(transaction):
    """Generate all unique subsequences of any length from the transaction with no duplicates."""
    subsequences = set()  # Use a set to store unique subsequences
    
    # Generate subsequences of all possible lengths (from 1 to len(transaction))
    for length in range(1, len(transaction) + 1):
        for subseq in combinations(transaction, length):
            # Ensure no duplicated items in the subsequence (set ensures uniqueness)
            if len(set(subseq)) == len(subseq):  # Check if subsequence has unique items
                subsequences.add(subseq)
    
    return subsequences
    
def count_all_combinations(transactions):
    """Count all subsequences (combinations) in transactions"""
    subseq_count = defaultdict(int)
    
    # Traverse each transaction
    for trans in transactions:
        print("Transaction:", trans)  # For debugging: print each transaction
        subsequences = generate_subsequences(trans)
        
        # Count each subsequence in the transaction
        for subseq in subsequences:
            subseq_count[subseq] += 1
            print(f"Subsequence: {subseq}")  # For debugging: print each subsequence found
    return subseq_count
    
def is_subsequence(subseq, trans):
    """Check if subseq is a subsequence of trans, preserving order"""
    it = iter(trans)
    return all(item in it for item in subseq)

def generate_candidate_subsequences(frequent_seqs, length):
    """Generate candidate subsequences of a given length from frequent subsequences"""
    candidates = set()
    for seq1 in frequent_seqs:
        for seq2 in frequent_seqs:
            # Only combine sequences of length (length - 1) to create length 'length' subsequences
            if len(seq1) == length - 1 and len(seq2) == length - 1:
                # Combine the sequences if the last item of seq1 matches the first item of seq2
                if seq1[-1] == seq2[0]:
                    candidates.add(seq1 + seq2[1:])
    return candidates

def count_subsequences(transactions, candidates):
    """Count the occurrences of subsequences in transactions"""
    subseq_count = defaultdict(int)
    for trans in transactions:
        for subseq in candidates:
            if is_subsequence(subseq, trans):
                subseq_count[subseq] += 1
    return subseq_count

def generalized_sequential_pattern_mining(raw_transactions, min_support):
    # Step 1: Preprocess transactions to extract only item values (simplify raw data)
    transactions = [[item for item in trans] for trans in raw_transactions]
    
    # Step 2: Find frequent 1-item subsequences (pairs as items)
    # single_items = defaultdict(int)
    # for trans in transactions:
    #     print(trans)
    #     for item in trans:
    #         single_items[(item,)] += 1
    #         print(item)

    single_items = defaultdict(int)

    for trans in transactions:
        # Generate all possible 2-item subsequences (pairs) without duplicates and unordered
        for pair in combinations(trans, 2):
            # Ensure no duplicate items in the pair
            if pair[0] != pair[1]:  # This check ensures no duplicate items in a pair
                single_items[frozenset(pair)] += 1  # Use frozenset to ignore order and handle unique pairs

    # Step 3: Filter 1-item subsequences by min_support
    F1 = {seq: count for seq, count in single_items.items() if count >= min_support}
    
    # Initialize frequent subsequences with F1
    frequent_subsequences = F1.copy()

    k = 2  # Starting with length 2 sequences

    # Loop to mine subsequences of increasing length k
    while F1:
        # Step 4: Generate candidate k-sequences from Fk-1 (previous frequent subsequences)
        candidate_k_seqs = generate_candidate_subsequences(list(F1.keys()), k)
        # print(k,list(F1.keys()))
        # print()
        # Step 5: Count occurrences of candidate k-sequences in transactions
        subseq_count = count_subsequences(transactions, candidate_k_seqs)
        
        # Step 6: Filter candidate subsequences by min_support
        Fk = {subseq: count for subseq, count in subseq_count.items() if count >= min_support}
        
        # Add Fk to the set of frequent subsequences
        frequent_subsequences.update(Fk)
        
        # If Fk is empty, stop the mining process
        if not Fk:
            break
        
        # Update F1 for the next iteration (Fk becomes Fk-1)
        F1 = Fk
        k += 1  # Increase subsequence length for the next iteration

    return frequent_subsequences

# Example raw transaction data (pairs of items)
raw_transactions= seq

# Set minimum support threshold
min_support = 100

# Run the Generalized Sequential Pattern Mining algorithm
frequent_subsequences = generalized_sequential_pattern_mining(raw_transactions, min_support)

# Output the results
# for subseq, count in frequent_subsequences.items():
#     print(f"Subsequence: {subseq}, Support: {count}")
print("Finished runnning GSP algo...")
sorted_subsequences = sorted(frequent_subsequences.items(), key=lambda x: x[1], reverse=True)

# Step 4: Save the result to a file
output_file = "frequent_subsequences_b.txt"
with open(output_file, "w") as file:
    # Write header
    file.write("Frequent 2-item Subsequences (Sorted by Support)\n")
    file.write("=" * 50 + "\n")
    
    # Write each subsequence and its support
    for subseq, count in sorted_subsequences:
        file.write(f"Subsequence: {subseq}, Support: {count}\n")

# Print a message to indicate that the file has been saved
print(f"Frequent subsequences saved to '{output_file}'")

Finished runnning GSP algo...
Frequent subsequences saved to 'frequent_subsequences_b.txt'


In [4]:
# sorted_subsequences = sorted(frequent_subsequences.items(), key=lambda x: x[1], reverse=True)

# # Step 4: Save the result to a file
# output_file = "frequent_subsequences.txt"
# with open(output_file, "w") as file:
#     # Write header
#     file.write("Frequent 2-item Subsequences (Sorted by Support)\n")
#     file.write("=" * 50 + "\n")
    
#     # Write each subsequence and its support
#     for subseq, count in sorted_subsequences:
#         file.write(f"Subsequence: {subseq}, Support: {count}\n")

# # Print a message to indicate that the file has been saved
# print(f"Frequent subsequences saved to '{output_file}'")